In [5]:
! ls

genetics  Jaminator


In [8]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/'My Drive'/'Projects'/'genetics'
! ls
! pip install pyeasyga

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Projects/genetics
cifar_net.py  genetic_net_generator.ipynb  genetics.py	__pycache__  README.md
  Created wheel for pyeasyga: filename=pyeasyga-0.3.1-py2.py3-none-any.whl size=6774 sha256=8859d73a6619a5cbbb389a7575b1592db8d07c3f32a25f135b0a404282fc0e64
  Stored in directory: /root/.cache/pip/wheels/ef/cf/ef/7aff9fcd6c1e59dc276182f29a32e7c197665dd5eb547f30e6
Successfully built pyeasyga


In [10]:
#################################################################
# DATASET LOADING
#################################################################
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

#load and prepare dataset
#can use standard python packages to load data as a numpy array, then
#convert the array to a torch.*Tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
#################################################################
#IMAGE SHOWER
#################################################################
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [0]:
#################################################################
#GENETIC ALGORITHM
#################################################################
from genetics import GeneticFit

validationloader = testloader

genetic = GeneticFit(trainloader, validationloader, max_iter = 5, pop_size = 3, num_generations = 2)
genetic.geneticFit()


number of layers is 2
number of layers is 1
number of layers is 2
[1,  2000] loss: 2.300
[1,  4000] loss: 2.286
[1,  6000] loss: 2.260
